In [1]:
!pip install requests beautifulsoup4 faiss-cpu numpy sentence-transformers transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 8.7 MB/s eta 0:00:

In [2]:
import os
output_dir = "/content/Legal_AEye-Opener"  # Use absolute path in Colab
os.makedirs(output_dir, exist_ok=True)
print(f"📂 Directory ensured: {output_dir}")


📂 Directory ensured: /content/Legal_AEye-Opener


In [3]:
import requests
from bs4 import BeautifulSoup
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import json
from transformers import pipeline
import re
import os
import chromadb

# Initialize models
model = SentenceTransformer("all-MiniLM-L6-v2")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

BASE_URL = "https://devgan.in"
LAW_TYPE = "bns"

# FAISS setup
d = 384  # Dimension of embeddings
index = faiss.IndexFlatL2(d)

# ChromaDB setup
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Ensure output directory exists
output_dir = "./Legal_AEye-Opener"
os.makedirs(output_dir, exist_ok=True)

# Single JSON storage
all_laws_data = []

def extract_sections(content):
    """Extracts sections while removing redundant section numbers/titles inside content."""
    sections = []

    # Remove "IPC Section <number>" references
    content = re.sub(r"IPC\s*Section\s*\d+\s*", "", content, flags=re.IGNORECASE)

    # Match "Section X –" format for BNS sections
    section_pattern = re.split(r"(?=Section\s+\d+\s*–)", content)

    for sec in section_pattern:
        match = re.match(r"Section\s+(\d+)\s*–\s*(.*)", sec.strip())

        if match:
            section_number = match.group(1)
            section_title = match.group(2)

            # Remove first occurrence of section number and title from content
            cleaned_content = re.sub(rf"^Section\s+{section_number}\s*–\s*{re.escape(section_title)}\s*", "", sec.strip(), count=1)

            sections.append({
                "section_number": section_number,
                "section_title": section_title,
                "section_content": cleaned_content.strip()  # Store cleaned content
            })

    return sections

def summarize_content(content):
    """Summarizes the given content."""
    return summarizer(content[:1024], max_length=150, min_length=50, do_sample=False)[0]['summary_text']

# Fetch the BNS chapters
MAIN_URL = f"{BASE_URL}/{LAW_TYPE}/"
response = requests.get(MAIN_URL)
soup = BeautifulSoup(response.text, "html.parser")

chapters = []
for row in soup.select("table.menu tr"):
    columns = row.find_all("td")
    if len(columns) == 2:
        chapter_number = columns[0].text.strip()
        chapter_title = columns[1].text.strip()
        chapter_link = BASE_URL + columns[1].find("a")["href"]
        chapters.append((chapter_number, chapter_title, chapter_link))

# Scrape each chapter and store in a single JSON
for chapter_number, chapter_title, chapter_link in chapters:
    print(f"📥 Fetching Chapter {chapter_number}: {chapter_title}")

    chapter_response = requests.get(chapter_link)
    chapter_soup = BeautifulSoup(chapter_response.text, "html.parser")

    content_div = chapter_soup.find("div", id="content")
    if not content_div:
        print(f"⚠️ Warning: Could not find content div in Chapter {chapter_number}. Trying fallback method...")
        content_div = chapter_soup.find("div", class_="main-content")

    chapter_content = content_div.get_text(separator="\n", strip=True) if content_div else "Content not found."

    # Extract sections
    sections = extract_sections(chapter_content)

    for section in sections:
        section_summary = summarize_content(section["section_content"])
        formatted_summary = (f"Section {section['section_number']} - {section['section_title']}: {section_summary}" if section["section_number"] != "Unknown" else section_summary)

        # Combine section title and content for embedding
        combined_text = section["section_title"] + " - " + section["section_content"]
        embedding = model.encode(combined_text).astype(np.float32)
        index.add(np.array([embedding]))  # Add to FAISS

        # Store in ChromaDB
        collection.add(
            ids=[f"{LAW_TYPE}_{chapter_number}_{section['section_number']}"],
            documents=[formatted_summary],
            metadatas=[{
                "law_type": LAW_TYPE.upper(),
                "chapter_number": chapter_number,
                "chapter_title": chapter_title,
                "section_number": section["section_number"],
                "section_title": section["section_title"],
                "section_summary": formatted_summary,
                "source_url": chapter_link
            }]
        )

        # Append to single JSON data
        all_laws_data.append({
            "law_type": LAW_TYPE.upper(),
            "chapter_number": chapter_number,
            "chapter_title": chapter_title,
            "section_number": section["section_number"],
            "section_title": section["section_title"],
            "section_content": section["section_content"],
            "section_summary": formatted_summary,
            "source_url": chapter_link
        })

        print(f"✅ Processed Chapter {chapter_number}, Section {section['section_number']}")

# Save all data to a single JSON file
json_output_path = os.path.join(output_dir, "bns_laws_data.json")
with open(json_output_path, "w", encoding="utf-8") as f:
    json.dump(all_laws_data, f, ensure_ascii=False, indent=4)

# Save FAISS index (backup)
faiss.write_index(index, os.path.join(output_dir, "bns_faiss.index"))

print(f"🎉 BNS Laws Scraping Completed! Data saved in: {json_output_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ Processed Chapter 19, Section 351
✅ Processed Chapter 19, Section 352
✅ Processed Chapter 19, Section 353


Your max_length is set to 150, but your input_length is only 134. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=67)


✅ Processed Chapter 19, Section 354
✅ Processed Chapter 19, Section 355


Your max_length is set to 150, but your input_length is only 142. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=71)


✅ Processed Chapter 19, Section 356
✅ Processed Chapter 19, Section 357
📥 Fetching Chapter 20: Repeal And Savings
✅ Processed Chapter 20, Section 358
🎉 BNS Laws Scraping Completed! Data saved in: ./Legal_AEye-Opener/bns_laws_data.json


In [4]:
import chromadb

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Check stored documents
documents = collection.get()

if documents and "ids" in documents and len(documents["ids"]) > 0:
    print(f"✅ Data is stored in ChromaDB. Total records: {len(documents['ids'])}")
else:
    print("❌ No data found in ChromaDB.")


✅ Data is stored in ChromaDB. Total records: 358


In [6]:
import chromadb

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Retrieve stored data
documents = collection.get()

# Check and print stored data
if documents and "ids" in documents and len(documents["ids"]) > 0:
    print(f"✅ Data is stored in ChromaDB. Total records: {len(documents['ids'])}\n")

    # Print each document with its metadata
    for i in range(len(documents["ids"])):
        print(f"🔹 ID: {documents['ids'][i]}")
        print(f"📄 Document Content: {documents['documents'][i]}")
        print("📝 Metadata:")
        for key, value in documents["metadatas"][i].items():
            print(f"   - {key}: {value}")
        print("\n" + "="*50 + "\n")  # Separator for readability
else:
    print("❌ No data found in ChromaDB.")


✅ Data is stored in ChromaDB. Total records: 358

🔹 ID: bns_01_1
📄 Document Content: Section 1 - Short title, commencement and application.: This Act may be called the Bharatiya Nyaya Sanhita, 2023. It shall come into force on such date as the Central Government may, by notification in the Official Gazette, appoint, and different dates may be appointed for different provisions. Every person shall be liable to punishment under this Act for every act or omission.
📝 Metadata:
   - chapter_number: 01
   - chapter_title: Preliminary
   - law_type: BNS
   - section_number: 1
   - section_summary: Section 1 - Short title, commencement and application.: This Act may be called the Bharatiya Nyaya Sanhita, 2023. It shall come into force on such date as the Central Government may, by notification in the Official Gazette, appoint, and different dates may be appointed for different provisions. Every person shall be liable to punishment under this Act for every act or omission.
   - section_title: S

In [16]:
from sentence_transformers import SentenceTransformer
import chromadb

# Load Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Connect to ChromaDB
chroma_client = chromadb.PersistentClient(path="./chroma_db")
collection = chroma_client.get_or_create_collection(name="bns_laws")

# Query ChromaDB
query = "theft"
query_embedding = model.encode(query).tolist()

# Search ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=5  # Return top 5 matches
)

# Check if results exist
if results and results.get('documents') and results['documents'][0]:
    print(f"✅ Data is stored in ChromaDB. Found {len(results['documents'][0])} matching records.\n")
    for doc, metadata in zip(results['documents'][0], results['metadatas'][0]):
        print(f"🔹 **Law Type**: {metadata.get('law_type', 'Unknown')}")
        print(f"📜 **Section**: {metadata.get('section_summary', 'No summary available')}")
        print(f"🔗 **Source**: {metadata.get('source_url', 'No source available')}")
        print("-" * 50)
else:
    print("❌ No matching legal sections found in ChromaDB.")


✅ Data is stored in ChromaDB. Found 5 matching records.

🔹 **Law Type**: BNS
📜 **Section**: Section 303 - Theft.: A person is said to cause a thing to move by removing an obstacle which prevented it from moving or by separating it from any other thing, as well as by actually moving it. Whoever, intending to take dishonestly any movable property out of the possession of any person without that person’s consent, moves that property in order to such taking.
🔗 **Source**: https://devgan.in/bns/chapter_17.php
--------------------------------------------------
🔹 **Law Type**: BNS
📜 **Section**: Section 309 - Robbery.: Theft is “robbery’ if the offender, at the time of committing the extortion, is in the presence of the person put in fear, and commits the extortion by putting that person in fear of instant death, of instant hurt, or of instant wrongful restraint. In all robbery there is either theft or extortion.
🔗 **Source**: https://devgan.in/bns/chapter_17.php
-----------------------------

chroma_client.delete_collection(name="bns_laws")


In [17]:
print(results)

{'ids': [['bns_17_303', 'bns_17_309', 'bns_17_305', 'bns_17_304', 'bns_17_307']], 'embeddings': None, 'documents': [['Section 303 - Theft.: A person is said to cause a thing to move by removing an obstacle which prevented it from moving or by separating it from any other thing, as well as by actually moving it. Whoever, intending to take dishonestly any movable property out of the possession of any person without that person’s consent, moves that property in order to such taking.', 'Section 309 - Robbery.: Theft is “robbery’ if the offender, at the time of committing the extortion, is in the presence of the person put in fear, and commits the extortion by putting that person in fear of instant death, of instant hurt, or of instant wrongful restraint. In all robbery there is either theft or extortion.', 'Section 305 - Theft in a dwelling house, or means of transportation or place of worship, etc.: Whoever commits theft, \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0in any building, tent or ves

In [18]:
import os

# Set your Groq API key (replace 'your_groq_api_key' with your actual API key)
os.environ["GROQ_API_KEY"] = "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"

# Verify if the key is set
print("Groq API Key Set:", "GROQ_API_KEY" in os.environ)


Groq API Key Set: True


In [19]:
%%capture
!pip install Groq

In [20]:
from groq import Groq

client = Groq()

# Convert the 'results' dictionary to a string before passing it to 'content'
results_string = str(results)  # or json.dumps(results) for better formatting

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "you are a helpful legal assistant and provide me a summarization in own language in points and explain all cases."
        },
        {
            "role": "user",
            "content": results_string,  # Use the string representation of 'results'
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature=0.5,
    max_completion_tokens=1024,
    top_p=1,
    stop=None,
    stream=False,
)

print(chat_completion.choices[0].message.content)

Here's a summary of the provided legal sections in my own language, broken down into points and explained for each case:

**Section 303 - Theft**

* This section defines theft as the act of moving a thing by removing an obstacle or separating it from another thing, with the intention of taking it dishonestly without the owner's consent.
* The key elements of theft are:
	+ The thing must be movable property.
	+ The person must intend to take it dishonestly.
	+ The person must move the property without the owner's consent.
* Example: If someone takes a book from a library without permission, they may be committing theft under this section.

**Section 309 - Robbery**

* This section defines robbery as a type of theft where the offender is in the presence of the person they are putting in fear.
* The key elements of robbery are:
	+ The offender must be in the presence of the person they are putting in fear.
	+ The offender must commit extortion by putting the person in fear of instant deat